In [6]:
import pandas as pd
import numpy as np
import argparse
import os
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve, auc, roc_curve, roc_auc_score
from tensorflow.keras import layers, models

In [2]:
model = models.load_model("../72model.h5")

In [3]:
X_test = pd.read_csv("../data/X_test.csv")
y_test = pd.read_csv("../data/y_test.csv")

In [ ]:
y_hat_p = model.predict(np.asarray(X_test).astype('float32'))
y_hat = np.argmax(y_hat_p,axis=1)
y_test = y_test['t'].values

In [18]:
np.sum(y_hat == y_test) / len(y_test)

0.7316655652603167

In [15]:
y_test

array([ 550,  164,  764, ...,  620, 1405,  850], dtype=int64)

In [11]:
# Get unique classes present in y_test
unique_classes = np.unique(y_test)

# Initialize list to store AUC values for each class
auc_values = []

# Iterate over each class
for c in unique_classes:
    # Get binary y_test for the current class
    y_test_binary = (y_test == c).astype(int)
    # Get predicted probabilities for the current class
    y_prob_class = y_hat_p[:, c]  # Assuming classes are 1-indexed
    # Calculate AUC for the current class
    auc = roc_auc_score(y_test_binary, y_prob_class)
    auc_values.append(auc)

# Calculate the mean AUC
mean_auc = np.mean(auc_values)

print("AUC for each class:", auc_values)
print("Mean AUC:", mean_auc)

AUC for each class: [0.9999968143228046, 0.9999519384448077, 0.9998101460554178, 0.9990829803272075, 0.9995813931457316, 0.9998152748582713, 0.9998564448428169, 0.9994862085030103, 0.9993490103995245, 0.9997394068841197, 0.9994410728461247, 0.9999763932678816, 0.9999999999999998, 0.9987983353373521, 0.9994948157645354, 0.999992037899598, 0.9999999999999999, 0.9992709948266425, 0.9998615411571974, 0.9996532618719804, 0.999532189353824, 0.9999877751109407, 0.9979160225963295, 0.9998994768816734, 0.9999840730093252, 0.9983558640847645, 0.9999590458595364, 0.9999653230228329, 0.9997850250007962, 0.9999918873976538, 0.9999764187916325, 0.9998148811656518, 0.9998274564447194, 0.9998822917028818, 0.9999940283296044, 0.999982748233021, 0.9998701615884886, 0.999942742997856, 0.9999936294066588, 0.9993154204854258, 1.0, 0.9999367554417258, 0.9998885305943708, 0.9999880566592088, 0.9999106233446513, 0.9998653035116637, 0.9999593923179821, 0.9999562087964586, 0.9990390470569498, 0.999996720652534,

In [41]:
from sklearn.metrics import precision_recall_curve, average_precision_score


unique_classes = np.unique(y_test)

auc_pr_values = []

# Iterate over each class
for c in unique_classes:
    # Get binary y_test for the current class
    y_test_binary = (y_test == c).astype(int)
    y_prob_class = y_hat_p[:, c]  
    auc_pr = average_precision_score(y_test_binary, y_prob_class)
    auc_pr_values.append(auc_pr)

mean_auc_pr = np.mean(auc_pr_values)

print("Area Under PR Curve for each class:", auc_pr_values)
print("Mean Area Under PR Curve:", mean_auc_pr)

Area Under PR Curve for each class: [0.9904928404928406, 0.5764259971803286, 0.3184557876646201, 0.042883580887075404, 0.7926184478252489, 0.184599429608504, 0.9700450107572597, 0.8196235164063093, 0.8163700362399725, 0.9292951822794013, 0.10463645722206985, 0.9614724109138963, 1.0, 0.07777034868685552, 0.15904254993796368, 0.6666666666666666, 1.0, 0.27035383248315625, 0.553103015528962, 0.11846667997066032, 0.11993773833656406, 0.9744473709359253, 0.031929105986926984, 0.20378787878787877, 0.9583520515124286, 0.05001892107994242, 0.9312401119630038, 0.9628759223223066, 0.029411764705882353, 0.9924790266935086, 0.9790986695257807, 0.1467391304347826, 0.9507901663856569, 0.956194063339372, 0.8541666666666666, 0.6652777777777777, 0.5712827223021233, 0.8482863950717237, 0.9590462835331763, 0.27169636588472745, 1.0, 0.8394929647864414, 0.10416666666666666, 0.7095238095238094, 0.7500032456785727, 0.1641965877259995, 0.3852564102564102, 0.14285714285714285, 0.08245193169886442, 0.98983142037

In [42]:
unique_classes

array([   0,    1,    2, ..., 1754, 1755, 1756], dtype=int64)

In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument("predictions", help="prediction result to evaluate")
# parser.add_argument("--test-name", "-n", help="name of test data set", nargs="*")
# parser.add_argument("--threshold", "-T", help="threshold for prediction", type=float, default=0.5)
# parser.add_argument("--no-threshold", "-N", help="performance evaluation without threshold (AUC,AUPR)", action="store_true")
# parser.add_argument("--evaluation-output","-o" , help="output for result evaluation")
# args = parser.parse_args()

In [27]:
no_th = True
th = .5
output_file = "output.csv"

if no_th:
   evaluation_df = pd.DataFrame(index=["Sen", "Spe", "Pre", "Acc", "F1","AUC", "AUPR"])
else:
   evaluation_df = pd.DataFrame(index=["Sen", "Spe", "Pre", "Acc", "F1"])

In [65]:
# confusion_matrix(y_test, y_hat).ravel()
# y_hat.shape == y_test.shape
# np.unique(y_test) -  np.unique(y_hat)
unique_classes = np.unique(np.concatenate([y_test, y_hat]))
confusion_matrix(y_test, y_hat,labels=unique_classes).ravel()


array([ 33,   0,   0, ...,   0,   0, 409], dtype=int64)

In [66]:
tn = fp = fn = tp = 0

# Iterate over each unique class
for c in unique_classes:
    true_positives = np.sum((y_test == c) & (y_hat == c))
    false_positives = np.sum((y_test != c) & (y_hat == c))
    false_negatives = np.sum((y_test == c) & (y_hat != c))
    true_negatives = np.sum((y_test != c) & (y_hat != c))

    tp += true_positives
    fp += false_positives
    fn += false_negatives
    tn += true_negatives

# Print TN, FP, FN, TP
print("True Negative (TN):", tn)
print("False Positive (FP):", fp)
print("False Negative (FN):", fn)
print("True Positive (TP):", tp)

True Negative (TN): 210718064
False Positive (FP): 33702
False Negative (FN): 33702
True Positive (TP): 91895


In [68]:
# tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
print("Evaluation of the set ")
sen = float(tp)/(fn+tp)
pre = float(tp)/(tp+fp)
spe = float(tn)/(tn+fp)
acc = float(tn+tp)/(tn+fp+fn+tp)
f1 = (2*sen*pre)/(sen+pre)
print( "\tSen : ", sen )
print("\tSpe : ", spe )
print("\tAcc : ", acc )
print("\tPrecision : ", pre)
print("\tF1 : ", f1)
result_dic = {"Acc": acc, "Sen" : sen, "Pre":pre, "F1":f1, "Spe":spe}
if no_th:
    fpr, tpr, thresholds_AUC = roc_curve(y_test, y_hat)
    AUC = auc(fpr, tpr)
    precision, recall, thresholds = precision_recall_curve(y_test,y_hat)
    AUPR = auc(recall,precision)
    print("\tArea Under ROC Curve(AUC): %0.3f" % AUC)
    print("\tArea Under PR Curve(AUPR): %0.3f" % AUPR)
    print("=================================================")
    result_dic.update({"AUC":AUC, "AUPR":AUPR}) 
evaluation_df = pd.Series(result_dic)
evaluation_output = output_file
if evaluation_output:
   print("save to %s"%output_file)
   dir_name, file_name = os.path.split(evaluation_output)
   if not os.path.isdir(dir_name):
      os.system("mkdir -p "+dir_name)
      print("No directory named %s : create directory" % dir_name)
   evaluation_df.to_csv(evaluation_output)


Evaluation of the set 
	Sen :  0.7316655652603167
	Spe :  0.9998400867492612
	Acc :  0.9996803639848246
	Precision :  0.7316655652603167
	F1 :  0.7316655652603167


ValueError: multiclass format is not supported